In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Imports

In [ ]:


from warnings import filterwarnings 
filterwarnings("ignore")
import os

import pandas as pd 
import numpy as np 
import polars as pl
from gc import collect 
from tqdm.notebook import tqdm
from scipy.optimize import minimize, Bounds

import kaggle_evaluation.default_inference_server

## Define Score and Investment range

In [ ]:
MIN_INVESTMENT, MAX_INVESTMENT = 0, 2

class ParticipantVisibleError(Exception): pass

def ScoreMetric(solution, submission, _):
    pos = submission['prediction']
    if pos.max() > MAX_INVESTMENT: raise ParticipantVisibleError(f'Max {pos.max()} > {MAX_INVESTMENT}')
    if pos.min() < MIN_INVESTMENT: raise ParticipantVisibleError(f'Min {pos.min()} < {MIN_INVESTMENT}')

    solution = solution.copy()
    solution['position'] = pos
    rfr, fwd = solution['risk_free_rate'], solution['forward_returns']
    strat_ret = rfr * (1 - pos) + pos * fwd

    excess_ret = strat_ret - rfr
    mean_excess = (1 + excess_ret).prod() ** (1 / len(solution)) - 1
    std = strat_ret.std()
    if std == 0: raise ZeroDivisionError

    sharpe = mean_excess / std * np.sqrt(252)
    strat_vol = std * np.sqrt(252) * 100
    market_vol = fwd.std() * np.sqrt(252) * 100
    market_mean = (1 + fwd - rfr).prod() ** (1 / len(solution)) - 1

    vol_penalty = 1 + max(0, strat_vol / market_vol - 1.2) if market_vol > 0 else 0
    return_penalty = 1 + ((max(0, (market_mean - mean_excess) * 100 * 252)) ** 2) / 100

    return min(sharpe / (vol_penalty * return_penalty), 1e6)


## Load data and define function

In [ ]:


train = pd.read_csv(
    f"/kaggle/input/hull-tactical-market-prediction/train.csv", 
    index_col = "date_id"
)

def fun(x):
    """
    Optimization logic adapted from Hull Tactical Kaggle discussion #608349.
    """
    ...


    solution   =  train[-180:].copy()
    submission =  pd.DataFrame({'prediction': x.clip(0, 2)}, index=solution.index)
    return - ScoreMetric(solution, submission, '')

x0  = np.full(180, 0.05)
res = minimize(fun, x0, method='Powell', bounds=Bounds(lb=0, ub=2), tol=1e-8)

print(res)

opt_preds = res.x

## Define Prediction to generate Allocations

In [ ]:
# Prepare predictions as a list
opt_pred_list = list(np.clip(opt_preds, 0, 2))  # Ensure bounds safety

def predict(test: pl.DataFrame) -> float:
    """
    Returns one prediction per call from the optimized allocation list.
    """
    if not opt_pred_list:
        raise IndexError("No more predictions left to serve.")

    pred = np.float64(opt_pred_list.pop(0))
    print(f"---> {pred:,.6f} | Remaining: {len(opt_pred_list)}")
    return pred


## Inference

In [ ]:
from kaggle_evaluation.default_inference_server import DefaultInferenceServer

inference_server = DefaultInferenceServer(predict)

# Serve predictions depending on environment
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()  # For official rerun
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))
